In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/bit_conference/

/content/gdrive/.shortcut-targets-by-id/1YDrmXvwQeDTF3AVegVo_-qlULY2-1-qE/bit_conference


### 이미지 생성

In [ ]:
!pip install diffusers --upgrade

In [ ]:
!pip install invisible_watermark transformers accelerate safetensors

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)
pipe.to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

...

unet%2Fconfig.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
import pandas as pd
import re

df_gen2 = pd.read_csv('coding/df_gen2.csv')

# id 칼럼에서 4자리 숫자 부분을 추출하는 함수
def filter_ids(id_str):
    match = re.search(r'(\d{4})$', id_str)  # id 끝 부분에서 숫자(0001~1000) 추출
    if match:
        return int(match.group(1)) <= 500  # 0001~0500까지만 남기기
    return False

# 필터링 적용
df_gen2 = df_gen2[df_gen2["id"].apply(filter_ids)].reset_index(drop=True)

In [ ]:
# 감정 리스트 정의
emotions = [
    "Exciting", "Hopeful", "Romantic",
    "Heartwarming", "Calm",
    "Sad", "Stress", "Lonely"
]

# 각 감정별로 데이터프레임을 생성하고 변수에 저장
for emotion in emotions:
    globals()[f"df_{emotion.lower()}"] = df_gen2[df_gen2["label"] == emotion].reset_index(drop=True)

In [ ]:
df_lonely = df_lonely.iloc[232:,]

In [ ]:
# 감정 리스트 정의
emotions = [
    "Exciting", "Hopeful", "Romantic",
    "Heartwarming", "Calm",
    "Sad", "Stress", "Lonely"
]

# Google Drive의 기본 저장 경로
base_dir = "/content/gdrive/MyDrive/bit_conference/image_gen"
os.makedirs(base_dir, exist_ok=True)  # 기본 디렉토리 생성 (없으면 생성)

# 각 감정별 데이터프레임을 순회하며 이미지 생성
for emotion in emotions:
    df_emotion = globals()[f"df_{emotion.lower()}"]  # 감정별 데이터프레임 가져오기
    save_dir = os.path.join(base_dir, emotion.lower())  # 감정별 디렉토리 설정
    os.makedirs(save_dir, exist_ok=True)  # 감정별 디렉토리 생성

    print(f"Processing {emotion}...")
    for index, row in tqdm(df_emotion.iterrows(), total=len(df_emotion), desc=f"Generating {emotion} Images"):
        # 원래 텍스트에 감정 정보 추가 (예: "with a romantic atmosphere")
        prompt = f"{row['text']} with a {emotion.lower()} atmosphere"
        image = pipe(prompt=prompt).images[0]  # 이미지 생성
        file_name = f"{row['id']}.png"  # id 칼럼을 파일명으로 사용
        file_path = os.path.join(save_dir, file_name)  # 파일 경로 설정
        image.save(file_path)  # 이미지 저장

print("모든 감정별 이미지 생성 및 저장 완료!")


Processing Lonely...


Generating Lonely Images:   0%|          | 0/268 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Generating Lonely Images:   0%|          | 1/268 [00:06<29:47,  6.70s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Generating Lonely Images: 100%|█████████▉| 267/268 [29:07<00:06,  6.50s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Generating Lonely Images: 100%|██████████| 268/268 [29:13<00:00,  6.54s/it]

모든 감정별 이미지 생성 및 저장 완료!


### 이미지 증강

In [ ]:
!pip install albumentations opencv-python

In [ ]:
emotions = ["exciting", "hopeful", "heartwarming", "calm"]
emotions = ["romantic", "sad", "stress", "lonely"]

In [ ]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from tqdm import tqdm  # tqdm 추가

# 기본 경로 설정
base_path = "/content/gdrive/MyDrive/bit_conference/image_gen/"
emotions = ["exciting", "hopeful", "heartwarming", "calm",
            "romantic", "sad", "stress", "lonely"]

# 증강 파이프라인 설정
transform = A.Compose([
    A.HorizontalFlip(p=0.5),        # 좌우 반전
    A.Rotate(limit=30, p=0.5),       # -30 ~ +30도 회전
    A.RandomBrightnessContrast(p=0.5),  # 밝기 및 대비 조절
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),  # 가우시안 블러
    A.HueSaturationValue(p=0.3),     # 색상 변경
    ToTensorV2()
])

# 증강 실행
for emotion in emotions:
    input_folder = os.path.join(base_path, emotion)  # 원본 이미지 폴더
    output_folder = os.path.join(base_path, f"{emotion}_aug")  # 증강 이미지 저장 폴더

    # 증강된 이미지 저장 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 원본 폴더 내 파일 리스트 가져오기
    image_files = [f for f in os.listdir(input_folder) if f.endswith(".png")]

    print(f"\nProcessing {emotion}: {len(image_files)} images")

    # tqdm을 사용하여 진행 상태 표시
    for img_name in tqdm(image_files, desc=f"Augmenting {emotion}", unit="img"):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)

        if image is None:
            print(f"이미지를 불러올 수 없습니다: {img_path}")
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR, 변환 필요

        # 3개의 증강 이미지 생성
        for i in range(3):
            augmented = transform(image=image)["image"]
            augmented = augmented.permute(1, 2, 0).numpy()  # Tensor를 Numpy로 변환
            augmented = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)  # 다시 BGR로 변환

            aug_img_name = f"{img_name.split('.')[0]}_aug_{i}.png"
            cv2.imwrite(os.path.join(output_folder, aug_img_name), augmented)

    print(f"증강 완료: {output_folder}")



Processing exciting: 423 images


Augmenting exciting: 100%|██████████| 423/423 [04:18<00:00,  1.64img/s]


증강 완료: /content/gdrive/MyDrive/bit_conference/image_gen/exciting_aug

Processing hopeful: 397 images


Augmenting hopeful: 100%|██████████| 397/397 [03:47<00:00,  1.75img/s]


증강 완료: /content/gdrive/MyDrive/bit_conference/image_gen/hopeful_aug

Processing heartwarming: 480 images


Augmenting heartwarming: 100%|██████████| 480/480 [04:01<00:00,  1.99img/s]


증강 완료: /content/gdrive/MyDrive/bit_conference/image_gen/heartwarming_aug

Processing calm: 446 images


Augmenting calm: 100%|██████████| 446/446 [03:38<00:00,  2.04img/s]

증강 완료: /content/gdrive/MyDrive/bit_conference/image_gen/calm_aug
